# Graph Library

### Warm up using Networkx library

In [9]:
import networkx as nx
G = nx.read_edgelist('utility/wikipedia_small.graph',create_using=nx.DiGraph())
print('Number of Nodes: {}'.format(G.number_of_nodes()))
p = nx.shortest_path(G,'1','2') 
print('Shortest path from 1 to 2: {}'.format(p))

Number of Nodes: 24166
Shortest path from 1 to 2: ['1', '752', '18618', '2']


In [10]:
import numpy as np
import scipy.sparse as sp
import heapq

Here we create a graph class containing three member functions -

- `shortest_path` uses Dijkstra's algorithm to calculate single source distance to all reachable nodes in the graph.

- `adjacency_matrix` generates the adjacency matrix of the graph.

- `pagerank` calculates the PageRank score of each nodes.

In [11]:
class Graph:
    
    def __init__(self):
        """ Initialize with an empty edge dictionary. """
        self.edges = {}
        
    def add_edges(self, edges_list):
        ''' Add a list of edges to the network. Use 1.0 to indiciate the presence of an edge. 
        
        Args:
            edges_list: list of (a,b) tuples, where a->b is an edge to add
        '''
        
        for i,j in edges_list:
            self.edges.setdefault(i,{})[j] = 1.0
            if j not in self.edges:
                self.edges[j] = {}
            
    def shortest_path(self, source):
        path, distance = {source:None}, {source:0}
        heap = [(0,source)]
        heapq.heapify(heap)
        while heap:
            wei, node = heapq.heappop(heap)
            for nxt in self.edges[node]:
                if wei + self.edges[node][nxt] < distance.get(nxt,float('inf')):
                    distance[nxt] = wei + self.edges[node][nxt]
                    path[nxt] = node
                    heapq.heappush(heap,(distance[nxt],nxt))
        
        return (distance, path)
    
    def adjacency_matrix(self):
        ''' Compute an adjacency matrix form of the graph.  
        
        Returns: tuple (A, nodes)
            A: a sparse matrix in COO form that represents the adjaency matrix
               for the graph (i.e., A[i,j] = 1 iff there is an edge i->j)
            nodes: a list of nodes indicating the node key corresponding to each
                   index of the A matrix
        '''
        nodes = list(self.edges)
        size = len(nodes)
        hashmap = {nodes[i] : i for i in range(size)}
        rows, cols = [], []
        for parent, children in self.edges.items():
            for nd in children:
                cols.append(hashmap[parent])
                rows.append(hashmap[nd])
        data = [1.0] * len(rows)
        adj_matrix = sp.coo_matrix((data,(rows,cols)),shape = (size,size))
        return (adj_matrix, nodes)
    
    def pagerank(self, d=0.85, iters=100):
        '''
        Compute PageRank score for each node in the graph.
        
        Args:
            damp: 1 - random restart factor
            iters: maximum number of iterations of power method
            
        Returns: 
        '''
        adj = self.adjacency_matrix()[0]
        n = adj.shape[0]
        deg_out_beta = adj.sum(0).T/d
        ranks = np.ones((n,1))/n
        for _ in range(iters):
            with np.errstate(divide='ignore'):
                new = adj.dot((ranks/deg_out_beta))
            new += (1-new.sum())/n
            ranks = new
        nodes = list(self.edges)
        hashmap = {nodes[i]:ranks[i][0,0] for i in range(len(nodes))}
        return hashmap

In [12]:
with open('utility/wikipedia_small.graph',encoding='utf8') as f:
    edge_list = []
    for line in f.readlines():
        start, end = line.split()
        edge_list.append((start,end))

In [13]:
g = Graph()
g.add_edges(edge_list)
dist, path = g.shortest_path('1')

__Compare result with networkx shortest path__

In [14]:
end = '2'
record = [end]
while end != '1':
    end = path[end]
    record.append(end)
print(record)

['2', '18618', '752', '1']


In [15]:
adj, nodes = g.adjacency_matrix()
%time pagerank = g.pagerank()

Wall time: 4.35 s


In [16]:
% time nx_page = nx.pagerank(G)

Wall time: 1min 53s


In [18]:
import operator
sorted(pagerank.items(), key=operator.itemgetter(1))[:5]

[('697', 6.626017534553544e-06),
 ('900', 6.626017534553544e-06),
 ('1707', 6.626017534553544e-06),
 ('1951', 6.626017534553544e-06),
 ('2042', 6.626017534553544e-06)]

In [19]:
import operator
sorted(nx_page.items(), key=operator.itemgetter(1))[:5]

[('697', 6.636980544652345e-06),
 ('900', 6.636980544652345e-06),
 ('1707', 6.636980544652345e-06),
 ('1951', 6.636980544652345e-06),
 ('2042', 6.636980544652345e-06)]

__Note:__ The PageRank algorithm from the networkx library is incredibly slow, and our self-made `pagerank` is almost 30 times faster and acquire similar results.